In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Dropdown, fixed
from IPython.display import display
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

In [2]:
# Path to go up one level from the current folder to the parent folder
parent_path = os.path.join(os.getcwd(), '..')

# Path to the target data folder
data_folder_path = os.path.join(parent_path, 'Data')

In [3]:
menu_df = pd.read_csv(os.path.join(data_folder_path, 'menu_analysis.csv'))
customers_df = pd.read_csv(os.path.join(data_folder_path, 'customers.csv'))
stores_df = pd.read_csv(os.path.join(data_folder_path, 'stores.csv'))
structure_df = pd.read_csv(os.path.join(data_folder_path, 'structure.csv'))
size_df = pd.read_csv(os.path.join(data_folder_path, 'order_size.csv'))
transactions_df = pd.read_csv(
        os.path.join(data_folder_path, "transactions.csv"),
        parse_dates=['Transaction_Date']  # Modify as needed if format issues arise
    )

### Modification 1

In [4]:
# Calculate profit per item
menu_df['Profit'] = menu_df['Price'] - menu_df['Cost']

In [5]:
menu_df.head()

,Group,Item_Number,Ingredient,Price,Cost,Points Earn,Points Redeem,Profit
0,Primary,Item_1,Chicken,6.5,2.0,65.0,400.0,4.5
1,Primary,Item_2,Beef,8.0,2.5,80.0,500.0,5.5
2,Primary,Item_3,Shrimp,7.5,3.0,75.0,600.0,4.5
3,Primary,Item_4,Tofu,5.5,1.0,55.0,200.0,4.5
4,Primary,Item_5,Chickpeas,4.5,0.5,45.0,100.0,4.0


### Modification 2

In [6]:
# Merge menu data with transaction data
merged_df = pd.merge(transactions_df, menu_df[['Item_Number', 'Ingredient', 'Profit']], on='Item_Number')

In [7]:
merged_df.head()

,Order_Number,Transaction_Date,Transaction_Time,Store_Number,Customer_ID,Item_Number,Ingredient,Profit
0,142292,2024-01-14,00:43:25,STR_7,CUST3706,Item_4,Tofu,4.5
1,134538,2023-07-22,03:11:53,STR_10,CUST2840,Item_4,Tofu,4.5
2,105529,2024-06-20,08:51:22,STR_8,CUST1036,Item_4,Tofu,4.5
3,34280,2024-05-06,17:40:34,STR_5,CUST0994,Item_4,Tofu,4.5
4,118343,2023-11-07,00:32:02,STR_5,CUST2229,Item_4,Tofu,4.5


### Modification 3

In [9]:
# Adding a default Quantity of 1 for demonstration purposes
merged_df['Quantity'] = 1

In [10]:
# Calculate total profit and quantity sold for each item
item_profitability = merged_df.groupby('Item_Number').agg({
    'Profit': lambda x: (x * merged_df.loc[x.index, 'Quantity']).sum(),
    'Quantity': 'sum',
    'Ingredient': 'first'
}).reset_index()

# Sort by total profit and get top 10
top_10_items = item_profitability.sort_values('Profit', ascending=False).head(10)

In [11]:
top_10_items

,Item_Number,Profit,Quantity,Ingredient
0,Item_1,301743.00,67054,Chicken
1,Item_2,221413.50,40257,Beef
11,Item_8,159976.00,39994,Lentils
8,Item_5,106480.00,26620,Chickpeas
9,Item_6,102289.00,24068,Pumpkin
7,Item_4,95508.00,21224,Tofu
2,Item_3,85432.50,18985,Shrimp
10,Item_7,80129.50,18854,Eggplant
3,Item_35,8457.75,2685,Soft Drink - XL
4,Item_36,7443.90,2757,Soft Drink - L


# 11. Top 10 Most Profitable Items

In [12]:
# Create the plot
fig = go.Figure()

# Bar chart for total profit
fig.add_trace(go.Bar(
    x=top_10_items['Ingredient'],
    y=top_10_items['Profit'],
    name='Total Profit',
    marker_color='lightgreen',
    hovertemplate='Item: %{x}<br>Total Profit: $%{y:.2f}<extra></extra>'
))

# Line chart for quantity sold
fig.add_trace(go.Scatter(
    x=top_10_items['Ingredient'],
    y=top_10_items['Quantity'],
    name='Quantity Sold',
    yaxis='y2',
    mode='lines+markers',
    line=dict(color='red', width=2),
    marker=dict(size=8),
    hovertemplate='Item: %{x}<br>Quantity Sold: %{y}<extra></extra>'
))

# Update layout
fig.update_layout(
    title='Top 10 Most Profitable Menu Items',
    xaxis=dict(title='Menu Item'),
    yaxis=dict(title='Total Profit ($)', side='left', showgrid=False),
    yaxis2=dict(title='Quantity Sold', side='right', overlaying='y', showgrid=False),
    legend=dict(x=1.1, y=1, bgcolor='rgba(255, 255, 255, 0.5)'),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

# Adjust hover info
fig.update_traces(
    hoverinfo="text",
    hoverlabel=dict(bgcolor="white", font_size=12),
    hovertemplate="Item: %{x}<br>" +
                  "Total Profit: $%{y:.2f}<br>" +
                  "Quantity Sold: %{customdata[0]}<br>" +
                  "Profit per Item: $%{customdata[1]:.2f}<extra></extra>",
    customdata=top_10_items[['Quantity', 'Profit']].div(top_10_items['Quantity'], axis=0)
)

# Rotate x-axis labels for better readability
fig.update_xaxes(tickangle=45)

fig.show()